In [2]:
import pandas as pd 
import numpy as np
import json
import os

In [4]:
# importando as infos de documentos

docs = pd.read_excel(r'.venv\Secretaria\UX_Documentos_02-01-2024.xlsx')

# importando financeiro

financial = pd.read_csv(r'.venv\Contratos\UX_Financeiro_02-01-2024.csv', sep=';')

# importando as notas

grades = pd.read_excel(r'.venv\Secretaria\UX_Notas_02-01-2024.xlsx')

In [36]:
# ajustando os dados do orbita para ter apenas os valores únicos e deixar por ordem de início

financial.sort_values(by=['Início em'], inplace=True)
financial.drop_duplicates(subset=['Cliente / E-mail'], keep='last', inplace=True)

In [38]:
# pegando as colunas certas para poder fazer a junção apenas do que importa:

list = [docs.columns, financial.columns, grades.columns]
list

[Index(['Aluno', 'Email', 'Curso', 'Documentação', 'Tipo 1', 'Situação 1',
        'URL 1', 'Tipo 2', 'Situação 2', 'URL 2', 'Tipo 3', 'Situação 3',
        'URL 3', 'Tipo 4', 'Situação 4', 'URL 4', 'Tipo 5', 'Situação 5',
        'URL 5'],
       dtype='object'),
 Index(['Contrato', 'Remove acesso', 'Status', 'Cobrança vigente',
        'Cobranças em aberto/atrasado', 'Cobranças reembolsadas',
        'Cobranças pagas', 'Cobranças futuras', 'Total de cobranças',
        'Primeira em aberto', 'Último pagamento', 'Valor da cobrança',
        'Total pago', 'Total futuro', 'Total em aberto/atrasado',
        'Total reembolsado', 'Valor adesão', 'Total da recorrência',
        'Início em', 'Cancelamento em', 'Produto', 'Nome produto',
        'Forma de pagamento', 'Cliente / Razão-Social', 'Cliente / E-mail',
        'Cliente / Fones', 'Cliente / Documento', 'Cliente / Endereço',
        'Cliente / Número', 'Cliente / Complemento', 'Cliente / Bairro',
        'Cliente / Cidade', 'Cliente /

In [47]:
# funcao para unificar as planilhas e pegar apenas as infos necessarias para operacao
# infos importantes
# Nome, email, apto, frequencia, media, qtd provas feitas, status financeiro


def concat_docs_grades_financial(docs, grades, financial):
    docs_grades = pd.merge(docs[['Email', 'Aluno', 'Documentação', 'Situação 1', 'Situação 2', 'Situação 3', 'Situação 4', 'Situação 5']], 
                           grades[['Turma', 'Nota', 'Prova','Email']], 
                           on='Email') 
        
    docs_grades_finacial = pd.merge(docs_grades, financial[['Total da recorrência', 'Cliente / E-mail', 
                                                'Início em', 'Cobranças pagas', 'Total pago', 'Status']], 
                                                left_on='Email', right_on='Cliente / E-mail', 
                                                how='left')
    docs_grades_finacial.drop('Cliente / E-mail', axis=1, inplace=True)
    return docs_grades, docs_grades_finacial
final_data_raw = concat_docs_grades_financial(docs, grades, financial)[1]

In [48]:
# verificacao final do dataframe para poder fazer as manipulações necessarias
# aqui voce precisa apenas trazer as colunas que vao ser utilizadas para poder montar em cima delas as informacoes importantes

final_data_raw.head()

,Email,Aluno,Documentação,Situação 1,Situação 2,Situação 3,Situação 4,Situação 5,Turma,Nota,Prova,Total da recorrência,Início em,Cobranças pagas,Total pago,Status
0,adoniaspimentajr@gmail.com,Adonias Pimenta Jr.,True,Válido,Válido,Válido,Válido,Válido,2022-01 - T01,8,M7. Times de UX de uma pessoa só,"10.746,00",08/01/2022 09:12:29,18.0,"10.746,00",Finalizado
1,adoniaspimentajr@gmail.com,Adonias Pimenta Jr.,True,Válido,Válido,Válido,Válido,Válido,2022-01 - T01,7,M9. Design Ops e Content Ops,"10.746,00",08/01/2022 09:12:29,18.0,"10.746,00",Finalizado
2,adoniaspimentajr@gmail.com,Adonias Pimenta Jr.,True,Válido,Válido,Válido,Válido,Válido,2022-01 - T01,7,"M16. Tópicos especiais - Tecnologias Modernas,...","10.746,00",08/01/2022 09:12:29,18.0,"10.746,00",Finalizado
3,adoniaspimentajr@gmail.com,Adonias Pimenta Jr.,True,Válido,Válido,Válido,Válido,Válido,2022-01 - T01,7,M17. Inteligência Emocional,"10.746,00",08/01/2022 09:12:29,18.0,"10.746,00",Finalizado
4,adoniaspimentajr@gmail.com,Adonias Pimenta Jr.,True,Válido,Válido,Válido,Válido,Válido,2022-01 - T01,7,Metodologias de Pesquisa II,"10.746,00",08/01/2022 09:12:29,18.0,"10.746,00",Finalizado


In [49]:
# Agregando por notas e transformando em média do aluno
# Aqui é importante não perder a informação do dataset completo para poder a gente fazer manipulações futuras com eles. 
# Sempre atribuir a variáveis novas para poder trazer as informações importantes 

def ux_notas(data):
    ux_notas = data.groupby(['Aluno', 'Turma'])['Nota'].mean().reset_index()
    ux_notas.rename(columns={'Nota':'Média'}, inplace=True)
    ux_notas['Média'] = ux_notas['Média'].round(2)
    return ux_notas

notas_ux = ux_notas(final_data_raw)
notas_ux

,Aluno,Turma,Média
0,Bárbara Wolf Lima,2023-08 - T04,9.67
1,ANA CAROLINA DEL VIGNA MISSE,2023-03 - T03,8.88
2,ANA LUIZA ANDRADE CHAGAS,2022-08 - T02,9.80
3,ARTHUR MEIRELES DOS SANTOS,2023-03 - T03,7.80
4,Adonias Pimenta Jr.,2022-01 - T01,8.00
...,...,...,...
525,victor aragão,2022-01 - T01,9.88
526,vinicius luiz miguel,2022-01 - T01,7.00
527,vitor de castro abreu,2022-08 - T02,8.13
528,waldir Palaoro Junior,2023-03 - T03,8.38


In [67]:
# Trazendo apenas aqueles que são concluintes e "em dia", independente da nota, documentação entregue ou não
# Todas as funções irão adicionar a filtagrem para poder formar a tabela final de trabalho

def ux_financial_ok(data):
    ux_ok = data.query('Status == ("Concluinte" | "Em dia")')
    ux_ok = pd.merge(ux_ok, ux_notas(data), on=['Aluno', 'Turma'], how='inner')
    return ux_ok

ux_financial = ux_financial_ok(final_data_raw)
ux_financial.head()

,Email,Aluno,Documentação,Situação 1,Situação 2,Situação 3,Situação 4,Situação 5,Turma,Nota,Prova,Total da recorrência,Início em,Cobranças pagas,Total pago,Status,Média
0,adylalira@gmail.com,Ádyla Hanne Lira Nascimento,False,Válido,Válido,Inválido,Inválido,Válido,2023-03 - T03,9,M5. Data Visualization - ao vivo com TSoM - Tu...,"8.046,00",06/03/2023 16:13:19,10.0,"4.470,00",Em dia,8.14
1,adylalira@gmail.com,Ádyla Hanne Lira Nascimento,False,Válido,Válido,Inválido,Inválido,Válido,2023-03 - T03,6,M4. Analytics UX e Estatística - Turma T3,"8.046,00",06/03/2023 16:13:19,10.0,"4.470,00",Em dia,8.14
2,adylalira@gmail.com,Ádyla Hanne Lira Nascimento,False,Válido,Válido,Inválido,Inválido,Válido,2023-03 - T03,8,M2 - Metodologias de Pesquisa - Turma T3,"8.046,00",06/03/2023 16:13:19,10.0,"4.470,00",Em dia,8.14
3,adylalira@gmail.com,Ádyla Hanne Lira Nascimento,False,Válido,Válido,Inválido,Inválido,Válido,2023-03 - T03,6,M1 - UX Research e Product Design - Turma T3,"8.046,00",06/03/2023 16:13:19,10.0,"4.470,00",Em dia,8.14
4,adylalira@gmail.com,Ádyla Hanne Lira Nascimento,False,Válido,Válido,Inválido,Inválido,Válido,2023-03 - T03,8,"M6. Ética, LGPD, Acessibilidade e Novas Tecnol...","8.046,00",06/03/2023 16:13:19,10.0,"4.470,00",Em dia,8.14


In [68]:
# Trocando "Documentação para Apto" e True | False para "Sim" e "Não"
ux_financial['Documentação'] = ux_financial['Documentação'].replace({True: 'Sim', False: 'Não'})
ux_financial.rename(columns={'Documentação':'Apto'}, inplace=True)

In [85]:
# Adicionando a frequencia do aluno de acordo com a quantidade de provas feitas
def ux_frequencia(data):
    ux_freq = data.groupby(['Aluno', 'Turma'])['Prova'].count().reset_index()
    ux_freq.rename(columns={'Prova':'Frequência'}, inplace=True)
    ux_freq['Frequência'] = ux_freq['Frequência'] / 18 * 100 
    ux_freq['Frequência'] = ux_freq['Frequência'].round(1)
    return ux_freq

ux_freq = ux_frequencia(ux_financial)
ux_freq.head()

,Aluno,Turma,Frequência
0,ANA CAROLINA DEL VIGNA MISSE,2023-03 - T03,44.4
1,Alessandra Hilda Gomes Resende,2022-08 - T02,77.8
2,Alessandra de Mello Camelo,2023-03 - T03,5.6
3,Alex Hobi Gomes,2023-03 - T03,27.8
4,Alexane Fernandes,2022-08 - T02,100.0


In [86]:
# Finalizando o dataframe final para exportação 
# Precisamos apenas tirar as provas e adicionar a frequência

ux_final = pd.merge(ux_financial, ux_freq, on=['Aluno', 'Turma'], how='inner')
ux_final.drop(['Prova', 'Nota'], axis=1, inplace=True)

In [89]:
# Finalizando apenas os que estão bem financeiramente 

ux_final.drop_duplicates(subset=['Aluno', 'Turma'], inplace=True)

In [92]:
# Exportando para excel 
ux_final.to_excel(r'C:\Users\Gabriel\Documents\Analize\Exportação\ux_final.xlsx', index=False)